In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
import numpy as np

In [3]:
L_smooth = 1 #np.inf
mu = 0.1

Inductance = 1
R = 1

n_func = 3
# solver = "ipopt" #mu=0.0001
solver = "ipopt_qcqp" #mu=0.1, 0.01, 0.001
# solver = "ipopt_qcqp_matrix"
# solver = "cvx_dccp"
# solver = "qcqp_sni"
# solver = "bisection_b"

# Ciropt problem

In [4]:
problem = co.admm_consensus(n_func, mu, L_smooth, R, Inductance)
problem.obj = problem.b + problem.d * 0.1 #0.5

# bounds = {  'alpha': {"ub": 0, "lb": 0},
#             'beta': {"ub": 1, "lb": 1},}

# res, sol = problem.solve(solver=solver, bounds=bounds, verbose=False)[:2]
res, model = problem.solve(solver=solver, verbose=False)[:2]
ca_vars = problem.vars

dim_G=12, dim_F=9
Ipopt total # of variables = 327
Actual # of variables = 177


In [5]:
res

{'b': 3.099999862512048,
 'h': 1.5499436435464504,
 'd': 9.368173033030048e-08,
 'alpha': -4.787083702178313e-12,
 'beta': -0.5610941517774929,
 'gamma': 1.860967617670214}

# PEP verification

In [6]:
res['b'] -= 0.1
assert res["b"] >= 0.1

In [7]:
problem = co.admm_consensus(n_func, mu, L_smooth, R, Inductance, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=-1.116473629333458e-12
